# chapter 9. chapter 9. Recommendation System

## Python Recommendation System Package - Surprise

### Surprise 패키지 소개 

콘텐츠 기반 필터링, 아이템 기반 필터링 그리조 잠재 요인 기반 협업 필터링은 최적화나 수행 속도 측면에서 좀 더 보완이 필요하다. <br>
**파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중 하나인 Surprise**는 사이킷런과 유사한 API와 프레임워크를 제공한다. <br>

In [12]:
!pip install scikit-surprise

**Surprise의 주요 장점**

- 다양한 추천 알고리즘
- Surprise 핵심 API는 사이킷런의 핵심 API와 유사한 이름으로 작성이 되어있다. 

### Surprise를 이용한 추천 시스템 구축 

추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤 행렬 분해를 통한 잠재 요인 협업 필터링을 수행한다. 

In [13]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

**Surprise에서 새로운 데이터 로딩은 Dataset Class를 이용해서만 가능**하다. <br>
**주요 데이터가 Row Level 형태로 되어 있는 포맷의 데이터만 처리**한다. 

In [14]:
data = Dataset.load_builtin('ml-100k')

# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값을 부여
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

과거 버전 데이터 파일의 분리 문자는 (\t)이다. <br>
Surprise에 사용자-아이템 평점 데이터를 적용할 떄 주의해야 할 점은 Row Level의 사용자-아이템 평점 데이터를 그대로 적용해야 한다는 것이다. <br>
Surprise는 자체적으로 Row Level의 데이터를 Column Level의 데이터로 변경하므로 원본이 Row Level 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 한다. <br>

**SVD 잠재 요인 협업 필터링 수행**

algo = SVD()와 같이 알고리즘 객체를 생성한 후 이 알고리즘 객체에 fit(학습 데이터 세트)를 수행해 학습 데이터 세트 기반으로 추천 알고리즘을 학습한다. 

In [15]:
algo = SVD(random_state = 0)
algo.fit(trainset)

**학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행**한다. <br>
Surprise에서 추천을 예측하는 메서드는 **test()** 와 **predict()** 이다. <br>
test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드이다. 즉, 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어준다. <br>
predict()는 개별 사용자와 영화에 대한 추천 평점을 반환한다. 

In [16]:
predictions = algo.test(testset)
print('prediction type : ', type(predictions), ' size : ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'>  size :  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.180754247821916, details={'was_impossible': False})]

'was_impossible'이 True이면 예측 값을 생성할 수 없는 데이터라는 의미이다. <br>

In [17]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측한다. <br>

In [18]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 한다. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


**테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가**한다. <br>

accuracy 모듈은 RMSE, MSE 등의 방법으로 추천 시스템의 성능 평가 정보를 제공한다. 

In [19]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

### Surprise 주요 모듈 소개 

**Dataset**
Surprise는 user_id, item_id, rating 데이터가 Row Level로 된 데이터 세트만 적용할 수 있다. 게다가 네 번째 Column부터는 아예 로딩을 수행하지 않는다. <br>

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**
Dataset.load_from_file API를 이용해 지정된 디렉터리에 있는 사용자-아이템 평점 데이터를 로딩한다. <br>
주의할 점은 로딩 되는 데이터 파일에 Column명을 가지는 Header 문자열이 있어서는 안된다는 점이다. <br>
to_csv() 함수를 이용해 간단히 이 Column Header를 삭제할 수 있다. 

**csv 파일로 사용자 평점 데이터 생성**

In [23]:
import pandas as pd

ratings = pd.read_csv('./data/ml latest small/ratings.csv') 
# ratings_noh.csv 파일로 업로드시 인덱스와 헤더를 모두 제거한 새로운 파일이 생성된다. 
ratings.to_csv('./data/ml latest small/ratings_noh.csv', index = False, header = False)

Dataset.load_from_file()을 적용하기 전에 Reader Class를 이용해 데이터 파일의 파싱 포맷을 정의해야 한다. <br>
Reader Class의 생성자에 각 필드의 Column 이름과 Column 분리 문자 그리고 최소-최대 평점을 입력해 객체를 생성하고, load_from_file()로 생성된 Reader 객체를 참조해 데이터 파일을 파싱하면서 로딩한다. <br>
Reader 설정이 완료되면, 이를 기반으로 데이터를 파싱하면서 Dataset을 로딩한다. 

**Reader Class로 파일의 포맷팅 지정하고 Dataset의 load_from_file()을 이용하여 데이터셋 로딩**

In [25]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('./data/ml latest small/ratings_noh.csv', reader = reader)

**Reader Class의 주요 생성 파라미터**

- line_format(string) : Column을 순서대로 나열한다. 입력된 문자열을 공백으로 분리해 Column으로 인식한다. 
- sep(char) : Column을 분리하는 분리자이며, '\t'이다. 판다스 DataFrame에서 입력받을 경우에는 기재할 필요가 없다.
- rating_scale(tuple, optional) : 평점 값의 최소-최대 평점을 설정한다. 기본값은 (1, 5)이지만 ratings.csv 파일의 경우 최소 평점이 0.5 최대 평점이 5이므로 (0.5, 5)로 설정

**학습과 테스트 데이터 세트로 분할하고 SVD로 학습한 후 테스트 데이터 평점 예측 후 RSME 평가** 

In [26]:
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors = 50, random_state = 0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

In [27]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./data/ml latest small/ratings.csv') 
reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야한다.  
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(n_factors = 50, random_state = 0)
algo.fit(trainset) 
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### Surprise 추천 알고리즘 클래스 

**Surprise에서 추천 예측을 위해 자주 사용되는 추천 알고리즘 Class**

- SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
- KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
- BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘

**Surprise SVD 비용 함수**는 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization을 적용한 것이다. <br>
이때 베이스라인이라는 의미는 각 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식을 말한다. 

### 베이스 라인 평점 

개인의 성향을 반영해 아이템 평가에 편향성(Bias) 요소를 반영해 평점을 부과하는 것을 **Baseline Rating**이라고 한다. <br>
보통 Baseline Rating은 **전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수** 공식으로 계산된다. 
- 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
- 사용자 편향 점수 : 사용자별 아이템 평점 평균 값 - 전체 평균 평점
- 아이템 편향 점수 : 아이템별 평점 평균 값 - 전체 평균 평점

### 교차 검증과 하이퍼 파라미터 튜닝 

Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate()와 GridSearchCV Class를 제공한다. <br>

**cross_validate()를 이용한 교차 검증**

In [28]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('./data/ml latest small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state = 0) 
cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8735  0.8720  0.8805  0.8780  0.8637  0.8736  0.0058  
MAE (testset)     0.6708  0.6696  0.6788  0.6763  0.6606  0.6712  0.0063  
Fit time          1.22    1.11    1.12    1.12    1.15    1.14    0.04    
Test time         0.47    0.15    0.26    0.14    0.14    0.23    0.13    


{'test_rmse': array([0.87354357, 0.87203293, 0.88045427, 0.8780446 , 0.86369724]),
 'test_mae': array([0.67081904, 0.66958573, 0.67877315, 0.6762546 , 0.66057719]),
 'fit_time': (1.2161409854888916,
  1.1144540309906006,
  1.1203410625457764,
  1.1198368072509766,
  1.1489109992980957),
 'test_time': (0.4669520854949951,
  0.15009474754333496,
  0.2580420970916748,
  0.13916683197021484,
  0.13915705680847168)}

**GridSearchCV 이용**

In [29]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8778201806004446
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

Surprise를 이용해 잠재 요인 협업 필터링 기반의 개인화된 영화 추천을 구현한다. <br>
**Surprise 패키지로 학습된 추천 알고리즘을 기반으로 특정 사용자가 아직 평점을 매기지 않은 영화 중에서 개인 취향에 가장 적절한 영화를 추천**한다. 

**SVD 학습은 TrainSet 클래스를 이용해야 한다.**

In [30]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생한다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

**DatasetAutoFolds를 이용한 전체 데이터를 TrainSet Class 변환**

In [31]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file = './data/ml latest small/ratings_noh.csv', reader = reader)

#전체 데이터를 학습데이터로 생성한다.  
trainset = data_folds.build_full_trainset()

In [33]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('./data/ml latest small/movies.csv')

# userId=9의 movieId 데이터 추출하여 movieId = 42 데이터가 있는지 확인 
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds == 42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 X')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 X
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [37]:
import surprise
print(surprise.__version__)

1.1.3


In [38]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

AttributeError: 'DatasetAutoFolds' object has no attribute 'to_inner_uid'

In [40]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:', len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


recomm_movie_by_surprise()는 추천 대상 영화 모두를 대상으로 추천 알고리즘 객체의 predict() 메서드를 호출하고 그 결과인 Prediction 객체를 리스트 객체로 저장한다. 

In [41]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있다.
    # [Prediction(uid = '9', iid = '1', est = 3.69), Prediction(uid = '9', iid = '2', est = 2.98)...]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
    predictions.sort(key = sortkey_est, reverse = True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n = 10)
print('Top-10 추천 영화 리스트')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


AttributeError: 'DatasetAutoFolds' object has no attribute 'to_inner_uid'